In [17]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split, LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [18]:
df = pd.read_csv('IphoneData.csv')
display = df['DISPLAY (inch)'].to_numpy().reshape(-1, 1)
storage = df['STORAGE (gb)'].to_numpy().reshape(-1, 1)
memory = df['MEMORY (gb)'].to_numpy().reshape(-1, 1)
cpu = df['CPU (cores)'].to_numpy().reshape(-1, 1)
gpu = df['GPU (cores)'].to_numpy().reshape(-1, 1)
neuralEngine = df['NEURAL ENGINE (cores)'].to_numpy().reshape(-1, 1)
weight = df['WEIGHT (gm)'].to_numpy().reshape(-1, 1)
frontCamera = df['FRONT CAMERA (mp)'].to_numpy().reshape(-1, 1)
battery = df['BATTERY (mAh)'].to_numpy().reshape(-1, 1)
numCameras = df['number of camera (back)'].to_numpy().reshape(-1, 1)
megapixels = df['Sum of megapixels'].to_numpy().reshape(-1, 1)
mainCamera = df['MAIN CAMERA (mp)'].to_numpy().reshape(-1, 1)
newdf = df.drop(columns=['NAME', 'CO2E'])
features = newdf.to_numpy()
emissions = df['CO2E'].to_numpy()

In [26]:
def modelScore(X, y, model):
    avgSSE = []
    avgMAPE = []
    splits = LeaveOneOut().get_n_splits(X)
    #cross validation using leave one out
    for i, (trainIndex, testIndex) in enumerate(LeaveOneOut().split(X)):
        X_train = X[trainIndex]
        X_test = X[testIndex]
        y_train = y[trainIndex]
        y_test = y[testIndex]
    

        if model == 'linear regression':
            fittedModel = LinearRegression().fit(X_train, y_train)
                
        elif model == 'decision tree':
            fittedModel = tree.DecisionTreeRegressor().fit(X_train, y_train)
                
        elif model == 'random forest':
            y_train = y_train.flatten()
            RF = RandomForestRegressor()#ccp_alpha=2.6, max_samples=.9, max_features='log2', min_samples_split=2,
                      #n_estimators=50, max_depth=17, min_samples_leaf=1, criterion='squared_error')
            fittedModel = RF.fit(X_train, y_train)
            
        elif model == 'support vector regression':
            y_train = y_train.flatten()
            fittedModel = SVR(kernel = 'linear').fit(X_train, y_train)
    
        elif model == 'polynomial regression':
            polyX_train = PolynomialFeatures(degree=2).fit_transform(X_train)
            polyX_test = PolynomialFeatures(degree=2).fit_transform(X_test)
            fittedModel = LinearRegression().fit(polyX_train, y_train)
            X_test = polyX_test

        elif model == 'xgboost':
            xgbModel = xgb.XGBRegressor()#learning_rate=0.225, n_estimators=225, booster='gblinear',
                                        #reg_alpha=0, reg_lambda=0, eval_metric='auc', 
                                        #objective='reg:squaredlogerror')
            fittedModel = xgbModel.fit(X_train, y_train)
        
        elif model =='lasso':
            fittedModel = Lasso().fit(X_train, y_train)
            

        y_pred = fittedModel.predict(X_test)
        
        #scores
        avgSSE.append(np.sum((y_test - y_pred) ** 2))
        avgMAPE.append(mean_absolute_percentage_error(y_test, y_pred))

    avgSSE, avgMAPE = np.mean(avgSSE), np.mean(avgMAPE)

    """#print results
    print("SSE = " + str(avgSSE))
    print("MAPE = " + str(avgMAPE))
    print(r2_score([1], [2]))
    print results
    print(model)
    print("R2 = " + str(avgR2))
    print("SSE = " + str(avgSSE))
    print("MAPE = " + str(avgMAPE))"""
    print()
    print((avgSSE))
    print((avgMAPE))

In [23]:
modelsList = ['linear regression', 'decision tree', 'random forest', 'support vector regression', 'polynomial regression', 'xgboost', 'lasso']


In [27]:
#display
for model in modelsList:
    (modelScore(display, emissions, model))


224.8655165085136
0.15501671316538138

236.59947945318822
0.15739054126823263

234.19227689239224
0.1566796730337723

257.5384205151984
0.1488394869956919

230.55200930510696
0.15570511420987973

212.10525814013485
0.14626280768857272

227.85898266061517
0.15754609280960108


In [30]:
#storage
for model in modelsList:
    (modelScore(storage, emissions, model))


83.50845209271095
0.0999480329887582

95.11931722082734
0.10618211537559613

96.3342984590608
0.10775380283236802

82.50034332152921
0.09799335128152331

85.72893046389952
0.10106046277785673

95.11868740357477
0.10618177721160053

83.50716293524184
0.09994847174638137


In [31]:
#memory
for model in modelsList:
    modelScore(memory, emissions, model)


254.86304062455747
0.16822449360198935

279.11171462339047
0.17862022407867273

275.00248461608146
0.17513578630272267

275.2860416666667
0.17192492846037233

264.54028072124066
0.1719295428505713

279.1103666373692
0.17861942294724545

255.38383498120962
0.16788413306084574


In [32]:
#cpu
for model in modelsList:
    (modelScore(cpu, emissions, model))


274.129470348574
0.17245779338661205

274.129470348574
0.17245779338661205

274.3688265268975
0.17251491208457956

280.5466666666668
0.16649211095248392

274.129470348574
0.17245779338661205

274.1294791747071
0.17245779713522813

274.129470348574
0.17245779338661205


In [33]:
#gpu
for model in modelsList:
    (modelScore(gpu, emissions, model))


258.09517808682523
0.16906445417586793

278.92567754098167
0.17736056274351272

276.61969927896746
0.17573408469609078

267.92666666666673
0.16432622897835272

267.6361901466014
0.1728998865155175

278.9258279681092
0.1773606179323092

260.145275200446
0.16859260195284143


In [34]:
#neural engine
for model in modelsList:
    (modelScore(neuralEngine, emissions, model))


285.3929328578102
0.17655364166586685

284.4340117026749
0.1780802477222687

287.77482986898264
0.17837951841522945

276.8332249149718
0.16613774974120313

284.4340117026752
0.17808024772226874

284.4334075317723
0.17808023047662466

282.624681778821
0.17543343531416947


In [35]:
#weight
for model in modelsList:
    (modelScore(weight, emissions, model))


202.7095466066277
0.14896365747459217

252.55280671296296
0.16886705063904614

241.96173142806
0.16513600978592008

236.36824863622144
0.15186237647988346

205.24683573372238
0.1498582253678992

252.5479665410112
0.1688656178030983

202.7095474553898
0.14901819775778394


In [39]:
#camera
for model in modelsList:
    (modelScore(mainCamera, emissions, model))


285.4511245416135
0.1759987431938411

285.4511245416135
0.17599874319384115

285.012794711538
0.17582290867003525

282.2000000000016
0.16736700422816508

285.4511245416135
0.1759987431938411

285.44854159664465
0.17599782602583236

284.1334099603724
0.1755161542110842


In [40]:
#camera
for model in modelsList:
    (modelScore(frontCamera, emissions, model))


220.0787165622331
0.15199491922811487

220.078716562233
0.15199491922811484

220.3966749456421
0.15215750316424434

228.70999999999935
0.14046497997001786

220.0787165622331
0.15199491922811484

220.07873453478078
0.15199511987467076

220.42767191294658
0.15251134086029708


In [41]:
#camera
for model in modelsList:
    (modelScore(numCameras, emissions, model))


177.6695241586945
0.14217392418476232

183.06569996419623
0.14541405678283736

182.99398751542506
0.1461343152258416

223.5645833333334
0.15166571899770756

183.06569996419617
0.14541405678283736

183.06571857864915
0.14541423420019878

179.72441154767412
0.1435886567392891


In [36]:
#camera
for model in modelsList:
    (modelScore(megapixels, emissions, model))


262.01795023438996
0.16905732855802444

188.20387885609125
0.145916362972798

187.29880881340728
0.1465282312277447

266.82048435944137
0.16935788489606587

228.14781706747638
0.15484773990350062

188.20368274574048
0.1459170039556894

262.0276674699176
0.1690049408047631


In [42]:
#battery
for model in modelsList:
    (modelScore(battery, emissions, model))


237.64374703816353
0.1621547169354084

246.1401909722222
0.16428590393711145

234.35173294609285
0.16366743751727394

230.76559978478016
0.14671728994522337

235.90905909417816
0.15885529700663722

240.68666870228662
0.16105945358735094

237.6440041439488
0.16215558642417108
